# Content-based recommendations (using cosine similarity)
This notebook shows the process used to calculate recommendations for user based on their previous ratings and in features extracted from content texts available in +TV4E platform.

Library requirements:
* Pandas
* NumPy
* Scikit-learn 
* NLTK (w/ portuguese stop words)

In [18]:
# for Python 2: use print only as a function
from __future__ import print_function

### Part 1 - Examine the data

In [19]:
# read CSV using a relative path
import pandas as pd
import numpy as np
import requests
URL_RATINGS='http://api_mysql.tv4e.pt/api/recommendations/ratings'
data=requests.get(URL_RATINGS)
df_ratings=pd.DataFrame(data.json())
URL_VIDEOS='http://api_mysql.tv4e.pt/api/recommendations/videos'
data= requests.get(URL_VIDEOS)
df_videos=pd.DataFrame(data.json())

In [20]:
# show dataframe first rows
df_videos.head()

,video_asgie_id,video_asgie_title_en,video_asgie_title_pt,video_date_creation,video_desc,video_id,video_location,video_title
0,3,Financial Services,Finanças,2017-09-26 15:21:16,Seja qual for o cenário para os escalões do IR...,4049,,As “habilidadezinhas” do Governo para descer o...
1,1,Health Care and Welfare Services,Saúde e Bem-Estar,2017-09-26 15:36:20,Mais pessoas recorrem aos centros de saúde par...,4051,,Deixar de fumar
2,1,Health Care and Welfare Services,Saúde e Bem-Estar,2017-09-26 15:38:36,Quase sete mil unidades de medicamentos apreen...,4053,,Medicamentos falsificados
3,1,Health Care and Welfare Services,Saúde e Bem-Estar,2017-09-26 15:40:12,Ministro da Saúde participa no encontro em Coi...,4054,,Conferência Mundial de Saúde Urbana
4,1,Health Care and Welfare Services,Saúde e Bem-Estar,2017-09-26 15:40:19,Rastreio visa despiste da ambliopia em criança...,4055,,CHTS | Saúde visual infantil


In [21]:
# show dataframe first rows
df_ratings.head()

,rating_date_creation,rating_value,user_id,video_id,video_watch_time,video_watched_type
0,2017-12-04 15:56:42,NaN,1,4453,100,forced
1,2017-12-04 17:20:39,-1.0,1,4454,16,forced
2,2017-12-04 15:07:13,NaN,1,4503,18,forced
3,2017-12-04 12:25:04,NaN,1,4545,17,forced
4,2017-12-04 15:08:26,1.0,1,4569,11,forced


In [22]:
df_ratings[['user_id', 'video_id', 'rating_value']]

,user_id,video_id,rating_value
0,1,4453,NaN
1,1,4454,-1.0
2,1,4503,NaN
3,1,4545,NaN
4,1,4569,1.0
5,1,4697,NaN
6,1,4705,NaN
7,1,4707,1.0
8,1,4709,NaN
9,1,4717,-1.0


In [23]:
#show the shape of dataframe
df_ratings.shape

(38, 6)

In [24]:
# show overall statistics of the dataframe
df_ratings.describe()

,rating_value,user_id,video_id,video_watch_time
count,21.000000,38.000000,38.000000,38.000000
mean,0.285714,5.631579,4687.710526,47.973684
std,0.845154,5.701688,86.941082,43.711268
min,-1.000000,1.000000,4453.000000,1.000000
25%,0.000000,1.000000,4699.000000,8.250000
50%,1.000000,4.000000,4721.000000,25.500000
75%,1.000000,8.000000,4740.250000,100.000000
max,1.000000,20.000000,4753.000000,100.000000


In [25]:
# filter ratings from user 1 and show statistics
df_ratings_user = df_ratings[(df_ratings.user_id==1)]
df_ratings_user.describe()

,rating_value,user_id,video_id,video_watch_time
count,6.000000,13.0,13.000000,13.000000
mean,0.000000,1.0,4632.307692,15.615385
std,1.095445,0.0,109.412358,26.209217
min,-1.000000,1.0,4453.000000,1.000000
25%,-1.000000,1.0,4545.000000,3.000000
50%,0.000000,1.0,4705.000000,9.000000
75%,1.000000,1.0,4717.000000,17.000000
max,1.000000,1.0,4722.000000,100.000000


### Part 2 - Clean ratings data

In [26]:
# rtemoving unused columns
df_ratings.drop('video_watched_type', axis=1, inplace=True)
df_ratings.drop('rating_date_creation', axis=1, inplace=True)
df_ratings

,rating_value,user_id,video_id,video_watch_time
0,NaN,1,4453,100
1,-1.0,1,4454,16
2,NaN,1,4503,18
3,NaN,1,4545,17
4,1.0,1,4569,11
5,NaN,1,4697,9
6,NaN,1,4705,3
7,1.0,1,4707,4
8,NaN,1,4709,5
9,-1.0,1,4717,1


In [44]:
# calculate implicit and explicit ratings
# XXX use a function to calculate implicit rating considering the video lead time
df_ratings['rating_implicit'] = (df_ratings['video_watch_time']/100) * 0.3
df_ratings['rating_explicit'] = (df_ratings['rating_value'])         * 0.7

# If the explicit rating was negative, the implicit will be negative
df_ratings.loc[(df_ratings.rating_explicit < 0), 'rating_implicit'] = df_ratings['rating_implicit'] * -1

# create a new column to put implicit or explicit rating rating_value
df_ratings['overall_rating_value'] = df_ratings['rating_implicit'] + df_ratings['rating_explicit']

# implicit rating is the watched time / explicit rating is the like-0-dislike
df_ratings['rating_implicit'] = df_ratings['video_watch_time']/100
df_ratings['rating_explicit'] = df_ratings['rating_value']
df_ratings['overall_rating_value']=df_ratings['overall_rating_value'].fillna(0)


# Right now, the overall rating will be NONE/NaN if no explicit rating was set
# So, we just consider the implicit rating if the user has seen at least 25% of the video
df_ratings.loc[ (df_ratings['overall_rating_value'] == 0) & (df_ratings['video_watch_time'] > 25),'overall_rating_value'] = (df_ratings['video_watch_time']/100) * 0.5
df_ratings.loc[ (df_ratings['overall_rating_value'] == 0) & (df_ratings['video_watch_time'] < 25),'overall_rating_value'] = (df_ratings['video_watch_time']/100 - 1) * 0.5

df_ratings


,rating_value,user_id,video_id,video_watch_time,rating_implicit,rating_explicit,overall_rating_value
0,NaN,1,4453,100,1.00,NaN,0.500
1,-1.0,1,4454,16,0.16,-1.0,-0.748
2,NaN,1,4503,18,0.18,NaN,-0.410
3,NaN,1,4545,17,0.17,NaN,-0.415
4,1.0,1,4569,11,0.11,1.0,0.733
5,NaN,1,4697,9,0.09,NaN,-0.455
6,NaN,1,4705,3,0.03,NaN,-0.485
7,1.0,1,4707,4,0.04,1.0,0.712
8,NaN,1,4709,5,0.05,NaN,-0.475
9,-1.0,1,4717,1,0.01,-1.0,-0.703


In [28]:
ratings_matrix = df_ratings.pivot_table(index=['user_id'], columns=['video_id'], values=['overall_rating_value'])
ratings_matrix                                     

overall_rating_value                                              \
video_id                 4453   4454  4503 4515   4545   4569   4574 4672   
user_id                                                                     
1                         0.5 -0.748  0.41  NaN  0.415  0.733    NaN  NaN   
3                         NaN    NaN   NaN  NaN    NaN    NaN    NaN  NaN   
4                         NaN    NaN   NaN  NaN    NaN    NaN    NaN  NaN   
5                         NaN    NaN   NaN  NaN    NaN    NaN    NaN  NaN   
6                         NaN    NaN   NaN  NaN    NaN    NaN    NaN  NaN   
7                         NaN    NaN   NaN  NaN    NaN    NaN    NaN  NaN   
8                         NaN    NaN   NaN  0.3    NaN    NaN  0.145  NaN   
9                         NaN    NaN   NaN  NaN    NaN    NaN    NaN  NaN   
20                        NaN    NaN   NaN  NaN    NaN    NaN    NaN  1.0   

                     ...                                                    \
video_id 4687   4697 ...   4736 4737   4738   4741   4743   4744 4747 4751   
user_id              ...                                                     
1         NaN  0.455 ...    NaN  NaN    NaN    NaN    NaN    NaN  NaN  NaN   
3         NaN    NaN ...    NaN  NaN    NaN    NaN    NaN    NaN  NaN  NaN   
4         1.0    NaN ...    NaN  NaN    NaN -0.703  0.495  0.057  NaN  NaN   
5         NaN    NaN ...    NaN  NaN    NaN    NaN    NaN    NaN  0.3  0.3   
6         NaN    NaN ...   0.19  1.0    NaN    NaN    NaN    NaN  NaN  NaN   
7         NaN    NaN ...    NaN  NaN    NaN    NaN    NaN    NaN  NaN  NaN   
8         NaN    NaN ...    NaN  NaN    NaN    NaN    NaN    NaN  NaN  1.0   
9         NaN    NaN ...    NaN  NaN  0.724    NaN    NaN  0.470  NaN  NaN   
20        NaN    NaN ...    NaN  NaN    NaN    NaN    NaN    NaN  NaN  NaN   

                      
video_id   4752 4753  
user_id               
1           NaN  NaN  
3           NaN  NaN  
4           NaN  NaN  
5         0.425  NaN  
6           NaN  NaN  
7           NaN  NaN  
8           NaN  NaN  
9           NaN  NaN  
20       -1.000  0.5  

[9 rows x 32 columns]

### Part 3 - Vectorize texts

In [29]:
# import and retrieve portuguese stop words
# stop words are not considered as token as usually they carry no meaning (!)
from nltk.corpus import stopwords
portuguese_stop_words = stopwords.words('portuguese')

In [30]:
# import and instantiate TF-IDF Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=2, max_df=0.5, stop_words=portuguese_stop_words)

In [31]:
# fit (train) the vectorizer with the corpus from video text contents
tfidf_matrix = tfidf.fit_transform(df_videos['video_desc'])
tfidf_matrix.toarray()

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [32]:
tfidf.get_feature_names()

['00',
 '00 horas',
 '000',
 '000 00',
 '000 euros',
 '000 utentes',
 '018',
 '018 2017',
 '06',
 '08',
 '09',
 '10',
 '10 10saúdealarga',
 '10 15',
 '10 aniversário',
 '10 anos',
 '10 dias',
 '10 horas',
 '10 meses',
 '10 mil',
 '10 milhões',
 '10 minutos',
 '10 novembro',
 '10 outubro',
 '100',
 '100 aquisição',
 '100 metros',
 '100 mil',
 '102',
 '102 mil',
 '10285',
 '10285 2017',
 '109',
 '10h30',
 '10h30 fundação',
 '10saúdealarga',
 '10saúdealarga elenco',
 '11',
 '11 27saúde',
 '11 anos',
 '11 novembro',
 '11 outubro',
 '110',
 '112',
 '112 acordo',
 '112 instituto',
 '112 recorda',
 '113',
 '113 2011',
 '118',
 '11h',
 '11h reforço',
 '11h00',
 '12',
 '12 000',
 '12 30',
 '12 anos',
 '12 dezembro',
 '12 meses',
 '12 mil',
 '12 outubro',
 '120',
 '120 km',
 '120 médicos',
 '1200',
 '125',
 '127',
 '129',
 '129 participantes',
 '12h00',
 '13',
 '13 milhões',
 '13 novembro',
 '13 outubro',
 '130',
 '131',
 '131 2017',
 '13h',
 '13h almoço',
 '14',
 '14 17',
 '14 mil',
 '14 novemb

### Part 4 - Build up user profile

In [33]:
# create dict video_id ==>> tfidf weights
tfidf_tokens = {}
tfidf_array = tfidf_matrix.toarray()
line_count = 0
for idx, row in df_videos.iterrows():
    tfidf_tokens[row.video_id] = tfidf_array[line_count]
    line_count += 1
tfidf_tokens

{4049: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4051: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4053: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4054: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4055: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4057: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4058: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4059: array([ 0.        ,  0.        ,  0.15365662, ...,  0.        ,
         0.        ,  0.        ]),
 4060: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4061: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4062: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4063: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4068: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4070: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4071: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4072: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4073: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4074: array([ 0.        ,  0.        ,  0.        , ...,  0.        ,
      

In [34]:
# define user ratings
user_ratings = df_ratings[(df_ratings.user_id==20)]
user_ratings

,rating_value,user_id,video_id,video_watch_time,rating_implicit,rating_explicit,overall_rating_value
34,1.0,20,4672,100,1.0,1.0,1.0
35,NaN,20,4721,100,1.0,NaN,0.5
36,-1.0,20,4752,100,1.0,-1.0,-1.0
37,NaN,20,4753,100,1.0,NaN,0.5


In [35]:
# created weighted user profile vector (dotproduct of vectors of items consumed and user ratings)
#   "In the original implementation, the profile was the sum of the item-tag vectors of all items 
#   the user has rated positively (>= 3.5 stars). This approach was later improved with weighted 
#   user profile (with the older implementation commented out for reference). Weighted profile is 
#   computed with weighted sum of the item vectors for all items, with weights being based on the 
#   user's rating."
#   See: http://eugenelin89.github.io/recommender_content_based/
user_profile = [0] * len(tfidf.get_feature_names())
for i in range(len(user_profile)):
    for idx, row in user_ratings.iterrows():
        user_profile[i] += row.overall_rating_value * tfidf_tokens[row.video_id][i]
    #user_profile = [v/len(user_ratings) for v in user_profile] # weight-ing user vector (?)
# normalize user profile vector
import numpy as np
user_profile = user_profile / np.linalg.norm(user_profile)
user_profile

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

### Part 5 - Calculate similarities between user profile and contents

In [36]:
# apply cosine similarity between user profile vector and content vectors
# See: http://eugenelin89.github.io/recommender_content_based/
from math import sqrt
def my_cosine_similarity(x, y):
    def square_rooted(v):
        return round(sqrt(sum([a * a for a in v])), 3)
   
    numerator = sum(a * b for a, b in zip(x, y))
    denominator = square_rooted(x) * square_rooted(y)
    return numerator/float(denominator)

estimated_user_ratings = {}
for video_id, token_weights in tfidf_tokens.items(): 
    if video_id not in user_ratings.video_id.values: # not calculating for contents already consumed 
        estimated_user_ratings[video_id] = my_cosine_similarity(user_profile, token_weights)
# order ratings
import operator
estimated_user_ratings = sorted(estimated_user_ratings.items(), key=operator.itemgetter(1))
estimated_user_ratings[:-10:-1]

[(4225, 0.15803743831597478),
 (4454, 0.13508244814832612),
 (4230, 0.11197615062103805),
 (4239, 0.10607655728586628),
 (4353, 0.10597446003285119),
 (4448, 0.10580566032594516),
 (4220, 0.098150034278477322),
 (4268, 0.089662280003604239),
 (4049, 0.086859208843875588)]

In [37]:
df_videos[(df_videos.video_id == 4148)].video_date_creation.values

array(['2017-10-03 10:31:30'], dtype=object)

In [38]:
# Using sklearn cosine_similarity (faster!)
from sklearn.metrics.pairwise import cosine_similarity
from math import sqrt
estimated_user_ratings = []
for video_id, token_weights in tfidf_tokens.items(): 
    # not calculating for contents already consumed
    if video_id not in user_ratings.video_id.values:
        estimated_user_ratings.append((
            video_id,
            df_videos[(df_videos.video_id == video_id)].video_date_creation.values[0],
            cosine_similarity([user_profile], [token_weights])[0]
        ))
#     if video_id not in user_ratings.video_id.values: # not calculating for contents already consumed 
#         estimated_user_ratings[video_id] = cosine_similarity([user_profile], [token_weights])
# order ratings
estimated_user_ratings = sorted(estimated_user_ratings, key=lambda tup: tup[2], reverse=True)[:12]
estimated_user_ratings

[(4225, '2017-10-11 22:11:59', array([ 0.15803744])),
 (4454, '2017-11-14 11:28:34', array([ 0.13508245])),
 (4230, '2017-10-12 16:17:34', array([ 0.11197615])),
 (4239, '2017-10-13 22:12:05', array([ 0.10607656])),
 (4353, '2017-10-23 09:11:21', array([ 0.10597446])),
 (4448, '2017-11-11 00:00:00', array([ 0.10580566])),
 (4220, '2017-10-11 09:11:42', array([ 0.09815003])),
 (4268, '2017-10-16 22:16:52', array([ 0.08966228])),
 (4049, '2017-09-26 15:21:16', array([ 0.08685921])),
 (4445, '2017-11-07 13:11:22', array([ 0.07792967])),
 (4296, '2017-10-19 22:13:58', array([ 0.07441095])),
 (4174, '2017-10-06 14:14:48', array([ 0.06569596]))]

In [39]:
# Using the dotproduct of user profile vector, content IDF and content vector
# See: https://www.analyticsvidhya.com/blog/2015/08/beginners-guide-learn-content-based-recommender-systems/
estimated_user_ratings = {}
for video_id, token_weights in tfidf_tokens.items():
    if video_id not in user_ratings.video_id.values: # removing contents already consumed 
        estimated_user_ratings[video_id] = 0
        for i in range(len(token_weights)):
            if token_weights[i] > 0:
                estimated_user_ratings[video_id] += user_profile[i] * tfidf.idf_[i] * token_weights[i]
# order ratings
import operator
estimated_user_ratings = sorted(estimated_user_ratings.items(), key=operator.itemgetter(1))
estimated_user_ratings[:-10:-1]

[(4225, 0.83611145568431244),
 (4230, 0.52236357972569747),
 (4454, 0.47060646825315672),
 (4239, 0.45623707710801875),
 (4448, 0.42532030444852376),
 (4220, 0.4209239208856016),
 (4353, 0.40145378778527607),
 (4049, 0.38573656591272187),
 (4268, 0.35094640629833873)]